## 오차제곱합

In [1]:
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [2]:
import numpy as np

# 정답 = 2, 원핫 인코딩으로 나타냄
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# example 1: '2'일 확률이 가장 높다고 추정
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [3]:
# example 2: '7'일 확률이 가장 높다고 추정
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

### 교차엔트로피

In [4]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta)) # 0이 되어 마이너스 무한대가 발생하지 않도록 아주 작은 값 delta 더해줌

In [5]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [6]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [7]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

### 미니배치 학습

In [8]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) # (60000, 784): 훈련데이터 60000개, 입력데이터 784개 (28*28 이미지 데이터)
print(t_train.shape) # (60000, 10)

(60000, 784)
(60000, 10)


In [9]:
# 무작위로 10장만
# np.random.choice(60000, 10): 0 이상 60000 미만의 수 중 무작위로 10개 골라냄
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

### (배치용) 교차 엔트로피 오차 구현하기

In [10]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y가 1차원이라면 데이터 하나당 reshape 함수로 데이터 형상 바꿔줌
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [11]:
# 정답 레이블이 원-핫 인코딩이 아니라 숫자 레이블로 주어졌을 때
def cross_entropy_error(y, t):
    if y.ndim == 1: # y가 1차원 = y의 출력 데이터가 1개 (=이미지 1장)
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
   
    batch_size = y.shape[0] # 행렬의 행 개수
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [12]:
t = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [13]:
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])

### 기울기

In [14]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [15]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원
        
    return grad

In [16]:
numerical_gradient(function_2, np.array([3.0, 4.0])) # (3, 4)에서의 기울기

array([6., 8.])

### 경사하강법

In [17]:
# f: 최적화하려는 함수
# init_x: 초깃값
# lr: learning rate
# step_num: 경사법에 따른 반복 횟수

def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

In [18]:
def function_2(x):
    return x[0]**2 + x[1]**2

import numpy as np
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

### 신경망에서의 기울기

In [19]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.rand(2, 3) # 정규분포로 초기화
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [20]:
net = simpleNet()
print(net.W) # 가중치 매개변수

[[0.44484738 0.83806527 0.78302425]
 [0.99459942 0.90887295 0.10804119]]


In [21]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[1.16204791 1.32082482 0.56705162]


In [22]:
np.argmax(p) # 최댓값의 인덱스

1

In [23]:
t = np.array([0, 0, 1]) # 정답 레이블
net.loss(x, t)

1.5969653991780897

In [24]:
def f(W): # 손실함수 계산하는 함수 f
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.22029324  0.25820051 -0.47849375]
 [ 0.33043986  0.38730077 -0.71774062]]


In [25]:
# 같은 함수 lamdba로 간단하게 쓰기
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

### 학습 알고리즘 구현하기

#### 2층 신경망 클래스 구현하기

In [26]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                          np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                          np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    # x: 입력 데이터, t: 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {} # 기울기 보관하는 딕셔너리
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) # 1번째 층 가중치의 기울기
        grads['b1'] = numerical_gradient(loss_W, self.params['b1']) # 1번째 층 편향 기울기
        grads['W2'] = numerical_gradient(loss_W, self.params['W2']) # 2번째 층 가중치의 기울기
        grads['b2'] = numerical_gradient(loss_W, self.params['b2']) # 2번째 총 편향 기울기
        
        return grads

In [27]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['W1'].shape # (784, 100)
net.params['b1'].shape # (100,)
net.params['W2'].shape # (100, 10)
net.params['b2'].shape # (10,)

(10,)

In [28]:
# 예측처리
x = np.random.rand(100, 784) # 더미 입력 데이터 (100장 분량)
y = net.predict(x)

In [29]:
x = np.random.rand(100, 784) # 더미 입력 데이터 (100장 분량)
t = np.random.rand(100, 10) # 더미 정답 레이블 (100장 분량)

grads = net.numerical_gradient(x, t) # 수치 미분 방식으로 매개변수 기울기 계산

grads['W1'].shape # (784, 100)
grads['b1'].shape # (100,)
grads['W2'].shape # (100, 10)
grads['b2'].shape # (10,)

#### 미니배치 학습 구현하기

In [30]:
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# 하이퍼파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num): #  10000번 반복
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산 (미니배치 대상 확률적 경사 하강법 수행)
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) # 성능 개선판!
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key] # 배열에 갱신값 추가
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

#### 시험 데이터로 평가하기

In [31]:
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num): #  10000번 반복
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산 (미니배치 대상 확률적 경사 하강법 수행)
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) # 성능 개선판!
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key] # 배열에 갱신값 추가
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | "
            + str(train_acc) + ", " + str(test_acc))